In [ ]:
# Imports and config
import sys
import os

notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root)

import torch
torch.backends.cudnn.benchmark = True

from src.data_loader import get_data_loaders
from src.train import train_model
from src.models.squeezenet import SqueezeNet
from src.models.mobilenet import MobileNetV1, MobileNetV2, MobileNetV3
from src.models.shiftnet import ShiftNet

# Configuration
DATA_DIR = "../data/FER2013"
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCHS = 40
LR = 1e-3

print("Device:", DEVICE)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Load data
print("\nLoading data...")
train_loader, test_loader = get_data_loaders(DATA_DIR, batch_size=BATCH_SIZE)
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Testing samples: {len(test_loader.dataset)}")

In [ ]:
# Prepare model save folders
os.makedirs("../models/mobilenet", exist_ok=True)
os.makedirs("../models/squeezenet", exist_ok=True)
os.makedirs("../models/shiftnet", exist_ok=True)

In [ ]:
# Train SqueezeNet from scratch
print("\nInitializing SqueezeNet model...")
squeezenet = SqueezeNet(num_classes=7, in_channels=1)

# Model summary
total_params = sum(p.numel() for p in squeezenet.parameters())
trainable_params = sum(p.numel() for p in squeezenet.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Train the model
print("\nStarting training...")
squeezenet_trained = train_model(
    squeezenet, 
    train_loader, 
    test_loader, 
    device=DEVICE,
    num_epochs=NUM_EPOCHS, 
    lr=LR
)

print("\nTraining complete!")

In [ ]:
# Train MobileNetV1 from scratch
print("\nInitializing MobileNetV1 model...")
mobilenetv1 = MobileNetV1(num_classes=7, in_channels=1)

# Print model summary
total_params = sum(p.numel() for p in mobilenetv1.parameters())
trainable_params = sum(p.numel() for p in mobilenetv1.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Start training
print("\nStarting training...")
mobilenetv1_trained = train_model(
    model=mobilenetv1,
    train_loader=train_loader,
    test_loader=test_loader,
    device=DEVICE,
    num_epochs=NUM_EPOCHS,
    lr=LR
)

print("\nTraining complete!")

In [ ]:
# Train MobileNetV2 from scratch
print("\nInitializing MobileNetV2 model...")
mobilenetv2 = MobileNetV2(num_classes=7, in_channels=1)

# Print model summary
total_params = sum(p.numel() for p in mobilenetv2.parameters())
trainable_params = sum(p.numel() for p in mobilenetv2.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Start training
print("\nStarting training...")
mobilenetv2_trained = train_model(
    model=mobilenetv2,
    train_loader=train_loader,
    test_loader=test_loader,
    device=DEVICE,
    num_epochs=NUM_EPOCHS,
    lr=LR
)

print("\nTraining complete!")

In [ ]:
from torchvision import models
import torch.nn as nn

# Load pretrained MobileNetV3 Small
model = models.mobilenet_v3_small(pretrained=True)

# Modify the first conv layer for 1-channel input
model.features[0][0] = nn.Conv2d(
    in_channels=1,
    out_channels=model.features[0][0].out_channels,
    kernel_size=model.features[0][0].kernel_size,
    stride=model.features[0][0].stride,
    padding=model.features[0][0].padding,
    bias=False
)

# Modify classifier for 7 classes
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 7)

trained_model = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    device=DEVICE,
    num_epochs=NUM_EPOCHS,
    lr=1e-4
)

In [ ]:
# Train ShiftNet from scratch
print("\nInitializing ShiftNet model...")
shiftnet = ShiftNet(num_classes=7)

# Model summary
total_params = sum(p.numel() for p in shiftnet.parameters())
trainable_params = sum(p.numel() for p in shiftnet.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Train the model
print("\nStarting training...")
shiftnet_trained = train_model(
    shiftnet, 
    train_loader, 
    test_loader, 
    device=DEVICE,
    num_epochs=NUM_EPOCHS, 
    lr=LR
)

print("\nTraining complete!")